# San Francisco Voter Guide Debate
This notebook uses the OpenAI assistant API to create a debate between a proponent and opponent on each issue in the 2024 San Francisco election. </br>
Each is seeded with the [official city election guide](https://voterguide.sfelections.org/local-ballot-measures). <br>
The debate instructions are included below for transparency. <br>
To change the measure and the number of rounds, adjust below. 


In [17]:
# a, b, c, d, e, f, g this year
issue = "a"
assert issue in ["a", "b", "c", "d", "e", "f", "g"]
# Debate
num_rounds = 3
debate_instructions="A debate is an organized argument or contest of ideas in which the \
    participants discuss a topic from two opposing sides.  Each side will show in an organized \
    and clever way why they believe to have the right answers.  They will use examples and \
    evidence to support their ideas while working towards a conclusion. \
    The aim of a debate is to convince the opposition that you are right. \
    When the two sides agree on the subject or when one side's arguments are more convincing \
    than the other side that is when the debate comes to a close. \
    Be concise and informative, do not repeat information already stated, and admit when you are wrong and when the other debater is right. \
    Respond to the Moderator's questions and to the other debater's arguments. "


# Setup
Some helper functions from the OpenAI Assistant cookbook: https://cookbook.openai.com/examples/assistants_api_overview_python

In [9]:
import time

def wait_on_run(run, thread, client):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

Download the appropriate issue

In [10]:
import os
import subprocess

os.makedirs("downloads", exist_ok=True)
if not os.path.exists(f"downloads/measure-{issue}.html"):
    subprocess.Popen(["wget", f"https://voterguide.sfelections.org/local-ballot-measures/measure-{issue}", "-O", f"downloads/measure-{issue}.html"])

# Debate
Create two assistants, one representing each side

In [11]:
from openai import OpenAI

client = OpenAI()

# Upload the file that you want
file = client.files.create(file=open(f"downloads/measure-{issue}.html", "rb"), purpose="assistants")
# Make your debaters
debater_1 = client.beta.assistants.create(
    name="Debater 1",
    instructions=debate_instructions + f"You support Measure {issue}. You are debating someone who oppose it. Preface each of your answers with Debater 1:",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id]
)
debater_2 = client.beta.assistants.create(
    name="Debater 2",
    instructions=debate_instructions + f"You oppose Measure {issue}. You are debating someone who supports it. Preface each of your answers with Debater 2:",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id]
)


As a user, act as the moderator and prompt the two

In [19]:
# Opening prompt, one thread for all messages
thread = client.beta.threads.create()

# Let em at it!
for i in range(num_rounds):
    if i == 0: 
        print("User: Start debate")
        message = client.beta.threads.messages.create(
            thread_id=thread.id, 
            role="user", 
            content = f"Moderator: Please concisely summarize Measure {issue} and your stance on it"
        )
    # Continue debate
    elif i < num_rounds - 1:
        print(f"User: Please respond")
        message = client.beta.threads.messages.create(
            thread_id=thread.id, 
            role="user", 
            content = f"Moderator: Please respond to the other debator's arguments and introduce novel arguments to make your case"
        )
    # Closing remarks
    else:
        print(f"User: Closing remarks")
        message = client.beta.threads.messages.create(
            thread_id=thread.id, 
            role="user", 
            content = f"Moderator: Please provide closing remarks on your stance on on Measure {issue}"
        )
    for debater in [debater_1, debater_2]:
        print(f"Round {i} for {debater.name}")
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=debater.id,
        )
        run = wait_on_run(run, thread, client)

User: Start debate
Round 0 for Debater 1
Round 0 for Debater 2


KeyboardInterrupt: 

# Results
Optionally export as an audio file using OpenAI's [TTS model](https://platform.openai.com/docs/guides/text-to-speech)

In [ ]:
os.makedirs("outputs", exist_ok=True)

# Print the results
messages = client.beta.threads.messages.list(thread_id=thread.id)
moderator, d1, d2 = "", "", ""
for msg in messages.data.reverse():
    if msg.assistant_id == debater_1.id:
        d1 += msg.content[0].text.value + "\n"
    elif msg.assistant_id == debater_2.id:
        d2 += msg.content[0].text.value + "\n"
    elif msg.role == "user" and msg.assistant_id == None:
        moderator += msg.content[0].text.value + "\n"

    print(msg.content[0].text.value)


moderator_path = "outputs" / "moderator.mp3"
moderator_response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=moderator
)
moderator_response.stream_to_file(moderator_path)

d1_path = "outputs" / "d1.mp3"
d1_response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=d1
)
d1_response.stream_to_file(d1_path)

d2_path = "outputs" / "d2.mp3"
d2_response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=d2
)
d2_response.stream_to_file(d2_path)

import pydub


